In [1]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [4]:
from functions import *

In [ ]:
df_zack = generate_zacks_data('LLY','2015-01-01','2023-01-31')
df_zack.head()

In [ ]:
df_zack['net_lterm_debt'] = df_zack['net_lterm_debt'].fillna(df_zack['tot_lterm_debt'])

df_zack['R'] = df_zack['ret_invst']*(df_zack['net_lterm_debt']+df_zack['mkt_val'])
df_zack.head()

In [ ]:
df_fc = fill_zacks_fc('LLY','2015-01-01','2023-01-31')
df_fr = fill_zacks_fr('LLY','2015-01-01','2023-01-31')
df_mktv = fill_zacks_mktv('LLY','2015-01-01','2023-01-31')
df_shrs = fill_zacks_shrs('LLY','2015-01-01','2023-01-31')

df_price = get_adj_close('LLY','2015-01-01','2023-01-31')

In [ ]:
df = df_price.join(df_zack,how='outer')

df['LLY_adj_close'] =  df['LLY_adj_close'].fillna(method='ffill')

In [ ]:
df[df['filing_date'].notna()]

In [ ]:
df_mkt_val = df_zack[['per_end_date','filing_date','mkt_val','net_lterm_debt','R']]
df_mkt_val = df_mkt_val.join(df['LLY_adj_close'])

In [ ]:
df_mkt_val

In [ ]:
df_mkt_val['date'] = df_mkt_val['filing_date']
df_mkt_val = df_mkt_val.set_index(['date'])
df_mkt_val = df_mkt_val[['mkt_val','LLY_adj_close','net_lterm_debt','R']]
df_mkt_val = df_mkt_val.rename(columns={'LLY_adj_close': 'adj_close'})
df_mkt_val

In [ ]:
df_mkt_val_daily = df[['LLY_adj_close']].join(df_mkt_val)
df_mkt_val_daily['mkt_val'] =  df_mkt_val_daily['mkt_val'].fillna(method='ffill')
df_mkt_val_daily['adj_close'] =  df_mkt_val_daily['adj_close'].fillna(method='ffill')
df_mkt_val_daily['net_lterm_debt'] =  df_mkt_val_daily['net_lterm_debt'].fillna(method='ffill')
df_mkt_val_daily['R'] =  df_mkt_val_daily['R'].fillna(method='ffill')
df_mkt_val_daily

In [ ]:
df_mkt_val_daily['mkt_val_daily'] = df_mkt_val_daily['mkt_val']*df_mkt_val_daily['LLY_adj_close'] /df_mkt_val_daily['adj_close'] 
df_mkt_val_daily.tail()

In [ ]:
df_mkt_val_daily.loc['2015-04-30':'2015-07-30']

In [ ]:
df_mkt_val_daily.loc['2022-02-23']

In [ ]:
df_mkt_val_daily['return_on_investment'] = df_mkt_val_daily['R']/(df_mkt_val_daily['net_lterm_debt']+df_mkt_val_daily['mkt_val_daily'])
df_mkt_val_daily.tail()

In [ ]:
df_mkt_val_daily.loc['2020-10-29']

In [ ]:
df_mkt_val_daily.loc['2020-12-31']

In [ ]:
df_mkt_val_daily.loc['2021-08-04']

In [83]:
def debt_to_market_cap(df,df_zack):

    df_tdte = df_zack[['per_end_date','filing_date','tot_debt_tot_equity']]

    df_tdte = df_tdte.join(df['adj_close'])   

    df_tdte['date'] = df_tdte['filing_date']
    df_tdte = df_tdte.set_index(['date'])
    df_tdte= df_tdte[['adj_close','tot_debt_tot_equity']]
    df_tdte = df_tdte.rename(columns={'adj_close': 'quarter_adj_close'})

    df_debt_to_market_cap = df[['adj_close']].join(df_tdte)

    df_debt_to_market_cap['quarter_adj_close'] =  df_debt_to_market_cap['quarter_adj_close'].fillna(method='ffill')
    df_debt_to_market_cap['tot_debt_tot_equity'] =  df_debt_to_market_cap['tot_debt_tot_equity'].fillna(method='ffill')

    df_debt_to_market_cap['debt_to_market_cap'] = df_debt_to_market_cap['tot_debt_tot_equity']*df_debt_to_market_cap['quarter_adj_close']/df_debt_to_market_cap['adj_close']

    return df_debt_to_market_cap


def return_on_investment(df,df_zack):

    df_zack['net_lterm_debt'] = df_zack['net_lterm_debt'].fillna(df_zack['tot_lterm_debt'])
    df_zack['R'] = df_zack['ret_invst']*(df_zack['net_lterm_debt']+df_zack['mkt_val'])

    df_mkt_val = df_zack[['per_end_date','filing_date','mkt_val','net_lterm_debt','R']]
    df_mkt_val = df_mkt_val.join(df['adj_close'])   

    df_mkt_val['date'] = df_mkt_val['filing_date']
    df_mkt_val = df_mkt_val.set_index(['date'])
    df_mkt_val = df_mkt_val[['mkt_val','adj_close','net_lterm_debt','R']]
    df_mkt_val = df_mkt_val.rename(columns={'adj_close': 'quarter_adj_close'})

    df_mkt_val_daily = df[['adj_close']].join(df_mkt_val)
    df_mkt_val_daily['mkt_val'] =  df_mkt_val_daily['mkt_val'].fillna(method='ffill')
    df_mkt_val_daily['quarter_adj_close'] =  df_mkt_val_daily['quarter_adj_close'].fillna(method='ffill')
    df_mkt_val_daily['net_lterm_debt'] =  df_mkt_val_daily['net_lterm_debt'].fillna(method='ffill')
    df_mkt_val_daily['R'] =  df_mkt_val_daily['R'].fillna(method='ffill')

    df_mkt_val_daily['mkt_val_daily'] = df_mkt_val_daily['mkt_val']*df_mkt_val_daily['adj_close'] /df_mkt_val_daily['quarter_adj_close'] 
    
    df_mkt_val_daily['return_on_investment'] = df_mkt_val_daily['R']/(df_mkt_val_daily['net_lterm_debt']+df_mkt_val_daily['mkt_val_daily'])

    df['return_on_investment'] = df_mkt_val_daily['return_on_investment']

    return df_mkt_val_daily

def price_to_earnings(df,df_zack):

    df_zack['eps_diluted_net'] = df_zack['eps_diluted_net'].fillna(df_zack['basic_net_eps'])
    df_zack.eps_diluted_net = np.where(df_zack.eps_diluted_net < 0, 0.001, df_zack.eps_diluted_net)

    df_pte = df_zack[['per_end_date','filing_date','eps_diluted_net']]  

    df_pte['date'] = df_pte['filing_date']
    df_pte= df_pte.set_index(['date'])
    df_pte = df_pte[['eps_diluted_net']]

    df_pte_daily = df[['adj_close']].join(df_pte)
    df_pte_daily['eps_diluted_net'] =  df_pte_daily['eps_diluted_net'].fillna(method='ffill')

    df_pte_daily['price_to_earnings'] = df_pte_daily['adj_close']/(df_pte_daily['eps_diluted_net'])

    return df_pte_daily

In [84]:
def calculate_ratio(ticker,start_date,end_date):

    df_zack = generate_zacks_data(ticker,start_date,end_date)
    df_price = get_adj_close(ticker,start_date,end_date)

    df = df_price.join(df_zack,how='outer')
    df['adj_close'] =  df['adj_close'].fillna(method='ffill')

    df1 = debt_to_market_cap(df,df_zack)
    df2 = return_on_investment(df,df_zack)
    df3 = price_to_earnings(df,df_zack)

    df['debt_to_market_cap'] = df1['debt_to_market_cap']
    df['return_on_investment'] = df2['return_on_investment']
    df['price_to_earnings'] = df3['price_to_earnings']

    return df[['debt_to_market_cap','return_on_investment','price_to_earnings']]


In [85]:
df = calculate_ratio('LLY','2014-09-30','2023-01-31')

In [86]:
df.loc[['2021-02-18','2021-03-31','2021-04-01']]

,debt_to_market_cap,return_on_investment,price_to_earnings
date,,,
2021-02-18,2.381655,7.910124,84.975909
2021-03-31,2.563957,8.509632,78.933966
2021-04-01,2.585826,8.581494,78.266395
